In [22]:
!pip install earthengine-api geemap --quiet
!pip install rasterio --quiet

In [271]:
import ee
import geemap.foliumap as geemap
import numpy as np
import rasterio
from rasterio.transform import from_origin

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import Concatenate

import requests
import pandas as pd
from datetime import datetime, timedelta

In [24]:
ee.Authenticate()
ee.Initialize()

Filtering Singapore's geographical boundary

In [73]:
singapore = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1") \
             .filter(ee.Filter.eq('ADM0_NAME', 'Singapore'))

Loading satellite data fromn Sentinel-1/2, MODIS

In [74]:
sentinel2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
    .filterBounds(singapore) \
    .filterDate('2024-01-01', '2025-04-22') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()


NDVI calc. from sentinel-2 (NIR-RED) / (NIR+RED)
-visible green within each pixel of image (high vegetation density, high NDVI)

In [113]:
ndvi = sentinel2.normalizedDifference(['B5', 'B4']).rename('NDVI')

palette = [ 'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301'] 

In [114]:
Map = geemap.Map(center=[1.3521, 103.8198], zoom=11)



Map.addLayer(ndvi, {'min': 0, 'max': 1, 'palette': palette }, 'NDVI')

Map

MODIS land surface temperature (MODIS/061/MOD11A2)

In [115]:
modis_lst = ee.ImageCollection("MODIS/061/MOD11A2") \
    .filterDate('2024-01-01', '2025-04-20') \
    .filterBounds(singapore) \
    .select('LST_Day_1km') \
    .mean() \
    .multiply(0.02).subtract(273.15) \
    .rename('LST') \
    .clip(singapore)# Convert to Celsius

Sentinel-1 for backscatter for land surface and urban analysis
Backscatter - measure of radar echo power returned by unit area of target's surface (localised measure of target surface reflects radar waves)

In [116]:
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(singapore) \
    .filterDate('2024-01-01', '2025-04-20') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('resolution_meters', 10)) \
    .select('VV') \
    .median().clip(singapore) \
    .rename('S1_VV')

In [117]:
Map.addLayer(sentinel1, {
    'min': -25,
    'max': 0,
    'palette': ['white', 'blue', 'black']
}, 'Sentinel-1 VV Median')


In [118]:
Map.addLayer(modis_lst, {'min': 20, 'max': 35, 'palette': ['blue', 'yellow', 'red']}, 'LST')
Map.addLayer(ndvi, {'min': 0, 'max': 0.5, 'palette': palette}, 'NDVI')



In [119]:
Map.addLayer(singapore, {}, 'Singapore Boundary')

In [120]:
Map

Creating a multi channel input sensors
Stacking

Stacking to multiband image

In [ ]:
stacked = sentinel1.addBands(ndvi).addBands(modis_lst)

Export as GeoTIFF

In [ ]:
import numpy as np

In [ ]:
region = singapore.geometry().bounds()

In [ ]:
image_np = geemap.ee_to_numpy(ee_object= stacked, region=region, scale=100)
#since i cant export to drive, export as numpy array

In [ ]:
print("Extracted image shape:", image_np.shape)
#ensure valid scale if not increase scale

Use of CNN in detecting spatial patterns such as urban heat clusters, flood prone zones
-CNN capture filters/kernal that helps us understand how pixels relate to their neighbours


Saving as GeoTIFF

In [ ]:
image_np = np.nan_to_num(image_np) 
#remove NaNs

In [ ]:
height, width, bands = image_np.shape
transform = from_origin(103.6, 1.47, 100, 100) 
#adjust long, lati to match sg

In [ ]:
with rasterio.open(
    'SG_stacked_input.tif', 'w',
    driver='GTiff',
    height=height,
    width=width,
    count=bands,
    dtype=image_np.dtype,
    transform=transform,
    crs='EPSG:4326'
) as dst:
    for i in range(bands):
        dst.write(image_np[:, :, i], i + 1)



In [ ]:
with rasterio.open('SG_stacked_input.tif') as src:
    image = src.read()  # Shape: (3, H, W)
    image = np.transpose(image, (1, 2, 0))  # Convert to (H, W, 3)

Normalize Each Band [0,1]

In [ ]:
#Sen1 (vv dB range: -25 to 0)
image[:, :, 0] = np.clip((image[:, :, 0] + 25) / 25, 0, 1)

In [ ]:
# NDVI is usually between -1 and 1
image[:, :, 1] = np.clip((image[:, :, 1] + 1) / 2, 0, 1)

In [ ]:
# LST (assumed 20°C to 40°C range)
image[:, :, 2] = np.clip((image[:, :, 2] - 20) / 20, 0, 1)

Slice image into 256x256 tiles which is 2.5km^2 at 10m resolution

In [ ]:
def tile_image(img, tile_size=256):
    h, w, c = img.shape
    tiles = []
    for i in range(0, h - tile_size + 1, tile_size):
        for j in range(0, w - tile_size + 1, tile_size):
            tile = img[i:i+tile_size, j:j+tile_size, :]
            tiles.append(tile)
    return np.array(tiles)

tiles = tile_image(image)
print("Tiled input shape:", tiles.shape)

Building model CNN Feature Extractor (spatial branch)

In [ ]:
input_shape = (256, 256, 3)

In [ ]:
cnn_input = tf.keras.Input(shape=input_shape, name='spatial_input')

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(cnn_input)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.GlobalAveragePooling2D()(x)
spatial_vector = layers.Dense(128, activation='relu', name='spatial_vector')(x)




In [ ]:
cnn_model = tf.keras.Model(inputs=cnn_input, outputs=spatial_vector, name="CNN_Spatial_Model")
cnn_model.summary()

Temporal forecasting (Rainfall - rain/ wind/ humidity, global horizontal irradiance)
rainfall daily- NEA
humdity daily-NEA
wind direction -NEA
wind speed - NEA
temperature? 

In [247]:
def get_rainfall_series(hours=120):
    base_url = "https://api.data.gov.sg/v1/environment/rainfall"
    timestamps = pd.date_range(end=datetime.utcnow(), periods=hours, freq='1H')

    all_data = []
    
    for ts in timestamps:
        iso_time = ts.strftime('%Y-%m-%dT%H:%M:%S')
        response = requests.get(base_url, params={'date_time': iso_time})
        if response.ok:
            data = response.json()
            readings = data.get('items', [{}])[0].get('readings', [])
            values = [r['value'] for r in readings if r.get('value') is not None]
            avg_rain = sum(values) / len(values) if values else 0
        else:
            avg_rain = 0  # fallback for missing API response
        all_data.append({'timestamp': ts, 'rainfall_mm': avg_rain})

    df = pd.DataFrame(all_data)
    return df


In [248]:
rainfall_df = get_rainfall_series()
rainfall_df.head()

,timestamp,rainfall_mm
0,2025-04-19 12:18:07.777032,0.018519
1,2025-04-19 13:18:07.777032,0.040741
2,2025-04-19 14:18:07.777032,0.427778
3,2025-04-19 15:18:07.777032,0.218519
4,2025-04-19 16:18:07.777032,0.000000


In [249]:
def get_humidity_series(hours=120):
    base_url = "https://api.data.gov.sg/v1/environment/relative-humidity"
    timestamps = pd.date_range(end=datetime.utcnow(), periods=hours, freq='1H')

    all_data = []
    
    for ts in timestamps:
        iso_time = ts.strftime('%Y-%m-%dT%H:%M:%S')
        response = requests.get(base_url, params={'date_time': iso_time})
        if response.ok:
            data = response.json()
            readings = data.get('items', [{}])[0].get('readings', [])
            values = [r['value'] for r in readings if r.get('value') is not None]
            avg_humidity = sum(values) / len(values) if values else 0
        else:
            avg_humidity = 0  # fallback for missing API response
        all_data.append({'timestamp': ts, 'humidity_percent': avg_humidity})

    df = pd.DataFrame(all_data)
    return df

In [250]:
humidity_df = get_humidity_series()
humidity_df.head()



,timestamp,humidity_percent
0,2025-04-19 12:18:18.057243,70.233333
1,2025-04-19 13:18:18.057243,75.455556
2,2025-04-19 14:18:18.057243,84.412500
3,2025-04-19 15:18:18.057243,86.555556
4,2025-04-19 16:18:18.057243,84.677778


In [251]:
def get_wind_speed_series(hours=120):
    base_url = "https://api.data.gov.sg/v1/environment/wind-speed"
    timestamps = pd.date_range(end=datetime.utcnow(), periods=hours, freq='1H')

    all_data = []

    for ts in timestamps:
        iso_time = ts.strftime('%Y-%m-%dT%H:%M:%S')
        response = requests.get(base_url, params={'date_time': iso_time})
        if response.ok:
            data = response.json()
            readings = data.get('items', [{}])[0].get('readings', [])
            values = [r['value'] for r in readings if r.get('value') is not None]
            avg_wind = sum(values) / len(values) if values else 0
        else:
            avg_wind = 0  # fallback for missing API response
        all_data.append({'timestamp': ts, 'wind_speed_mps': avg_wind})

    df = pd.DataFrame(all_data)
    return df


In [252]:
wind_df = get_wind_speed_series()
wind_df.head()

,timestamp,wind_speed_mps
0,2025-04-19 12:18:26.820308,4.000000
1,2025-04-19 13:18:26.820308,4.611111
2,2025-04-19 14:18:26.820308,3.055556
3,2025-04-19 15:18:26.820308,2.577778
4,2025-04-19 16:18:26.820308,2.444444


In [253]:
def get_temperature_series(hours=120):
    base_url = "https://api.data.gov.sg/v1/environment/air-temperature"
    timestamps = pd.date_range(end=datetime.utcnow(), periods=hours, freq='1H')

    all_data = []
    
    for ts in timestamps:
        iso_time = ts.strftime('%Y-%m-%dT%H:%M:%S')
        response = requests.get(base_url, params={'date_time': iso_time})
        if response.ok:
            data = response.json()
            readings = data.get('items', [{}])[0].get('readings', [])
            values = [r['value'] for r in readings if r.get('value') is not None]
            avg_temp = sum(values) / len(values) if values else 0
        else:
            avg_temp = 0  # fallback
        all_data.append({'timestamp': ts, 'temperature_celsius': avg_temp})
    
    df = pd.DataFrame(all_data)
    return df
    

In [254]:
temp_df = get_temperature_series()
temp_df.head()

,timestamp,temperature_celsius
0,2025-04-19 12:18:35.626713,31.277778
1,2025-04-19 13:18:35.626713,30.622222
2,2025-04-19 14:18:35.626713,26.966667
3,2025-04-19 15:18:35.626713,27.155556
4,2025-04-19 16:18:35.626713,28.122222


Preparing for LSTM model
(120,4)

Standardize all timestamps since jn it returns (0,5) and after we realised some timestamps failed to fetch......
so the issue is somewhere it runs 1 min faster/slower thus unable to merge......

In [260]:
for df in [rainfall_df, humidity_df, wind_df, temp_df]:
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('H')
    
print("Rainfall rows:", len(rainfall_df))
print("Humidity rows:", len(humidity_df))
print("Wind rows:", len(wind_df))
print("Temp rows:", len(temp_df))   

Rainfall rows: 120
Humidity rows: 120
Wind rows: 120
Temp rows: 120


In [261]:
df_merged = rainfall_df.merge(humidity_df, on='timestamp') \
                       .merge(wind_df, on='timestamp') \
                       .merge(temp_df, on='timestamp')

print("Any missing values?", df_merged.isnull().values.any())

missing_rows = df_merged[df_merged.isnull().any(axis=1)]
print(f"Rows with missing values: {len(missing_rows)}")
print(missing_rows[['timestamp'] + [col for col in df_merged.columns if 'value' in col or 'percent' in col]])
df_merged = df_merged.fillna(method='ffill').fillna(method='bfill')


In [262]:
df_merged.shape

(120, 5)

In [263]:
df_merged['rainfall_norm'] = df_merged['rainfall_mm'] / 100       # assuming 100 mm/hr as max
df_merged['humidity_norm'] = df_merged['humidity_percent'] / 100
df_merged['wind_norm']     = df_merged['wind_speed_mps'] / 15     # assume 15 m/s max wind
df_merged['temp_norm']     = (df_merged['temperature_celsius'] - 20) / 15  # assuming 20–35°C range

In [264]:
X_temporal = df_merged[['rainfall_norm', 'humidity_norm', 'wind_norm', 'temp_norm']].values
X_temporal.shape

(120, 4)

In [265]:
np.save('X_temporal_120h.npy', X_temporal)

Defining the LSTM Temporal Encoder 
time series of weather data over 120h
learns a compressed vector (embedding) that captures how weather is evolving over time
*memory unit


Long term short memory - Recurrent Neural network that processes one timestep (120h) at a time, while maintaining internal memory that gets updated as it gets new data.

At each hours -
reads 4 weather feature for that hour,
updates hidden state memory to reflect recent rainfall accum trends in temp/humidity/ wind accerleration or drops

128 dimensional context vector - summary of recent weather history to capture pattern

In [268]:
temporal_input = Input(shape=(120, 4), name='temporal_input')

In [269]:
# LSTM block
x = LSTM(128, return_sequences=False)(temporal_input)
x = Dropout(0.3)(x)
temporal_vector = Dense(128, activation='relu', name='temporal_vector')(x)


In [270]:
lstm_encoder = Model(inputs=temporal_input, outputs=temporal_vector, name='LSTM_Encoder')
lstm_encoder.summary()

Model: "LSTM_Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 temporal_input (InputLayer  [(None, 120, 4)]          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 128)               68096     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 temporal_vector (Dense)     (None, 128)               16512     
                                                                 
Total params: 84608 (330.50 KB)
Trainable params: 84608 (330.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Fusion with CNN spatial encoder with LSTM encoder
